In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abadi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abadi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import os
import string
import copy
import pickle

In [3]:
title = "20_newsgroups"
os.chdir("C:/20_newsgroups")
paths = [0]
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'):
    for i in filenames:
        paths.append(str(dirpath)+str("\\")+i)
print(dirpath)

C:\20_newsgroups/20_newsgroups/alt.atheism


In [4]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], '')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data


def stemming(data):
    stemmer= PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [5]:
def preprocess(data, query):
    if not query:
        data = remove_header(data) 
        data = convert_lower_case(data)
        data = convert_numbers(data)
        data = remove_punctuation(data)
        data = remove_stop_words(data)
        data = remove_apostrophe(data)
        data = remove_single_characters(data)
        data = stemming(data) 
    return data

In [6]:
doc = 0
postings = pd.DataFrame()

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    
    #Genrate matrex posting list
    if doc%100 == 0:
        print(doc)
    tokens = word_tokenize(str(preprocessed_text))
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            p.add(doc)
            postings[token][0] = p 
        else:
            postings.insert(value=[{doc}], loc=0, column=token)
    doc += 1 

#Save the output:
postings.to_pickle(title + "_unigram_postings")

No Header
0


In [7]:
postings

,imight,exam,ofit,acompil,side,uneven,qualityi,inpubsocreligionchristianotherscontradict,ftprutgersedu,mcdowelldjp,...,atheist,zero,version,two,nine,decemb,one,resourceslastmodifi,atheismresourcesaltatheismarchivenam,archivenam
0,{22},{22},{22},{22},{22},{22},{22},{22},{22},{22},...,"{1, 2, 6}","{1, 2, 3, 5, 6, 7, 11, 16, 18, 19, 20, 21, 22}","{1, 2, 3}","{1, 2, 3, 5, 6, 8, 18, 19, 20, 21}","{1, 2, 3, 5, 6, 18, 19, 20}",{1},"{1, 2, 3, 4, 5, 6, 8, 14, 15, 17, 18, 19, 20, ...",{1},{1},"{1, 2}"


In [8]:
postings['one'][0]

{1, 2, 3, 4, 5, 6, 8, 14, 15, 17, 18, 19, 20, 21, 22}

In [9]:
postings['exam'][0]

{22}

In [10]:
postings['nine'][0]

{1, 2, 3, 5, 6, 18, 19, 20}

In [11]:
s1 = postings['one'][0]
s2 = postings['nine'][0]
s3 = postings['exam'][0]
print(s1)
print(s2)
print(s3)

print('intersection = ', s1 & s2 & s3)

{1, 2, 3, 4, 5, 6, 8, 14, 15, 17, 18, 19, 20, 21, 22}
{1, 2, 3, 5, 6, 18, 19, 20}
{22}
intersection =  set()
